In [ ]:
import sys
!{sys.executable} -m pip install -q -U pip
!{sys.executable} -m pip install -q -U -r requirements.txt
!{sys.executable} -m pip install -q -U -r ./EPFD/requirements.txt
!{sys.executable} -m pip install -q -e ./EPFD/PyEnsemble

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import wandb
wandb.login()

run = wandb.init(project="htcv", entity="nicojahn")

In [ ]:
import time
attempt = 60*5

while attempt:
    try:
        time.sleep(1)
        label_artifact = run.use_artifact("nicojahn/htcv/validation_labels:latest", type="dataset")
        label_artifact_dir = label_artifact.download()
        break
    except ValueError:
        attempt -= 1
assert attempt > 0

In [ ]:
import time
attempt = 60*5

while attempt:
    try:
        time.sleep(1)
        data_artifact = run.use_artifact("nicojahn/htcv/validation_predictions:latest", type="dataset")
        data_artifact_dir = data_artifact.download()
        break
    except ValueError:
        attempt -= 1
assert attempt > 0

In [ ]:
from pathlib import Path

validation_labels = [str(p) for p in Path(label_artifact_dir).glob("*")]
assert len(validation_labels) == 1
validation_labels = validation_labels[0]
with open(validation_labels, "rb") as file:
    validation_labels = pickle.load(file)


validation_predictions = [str(p) for p in Path(data_artifact_dir).glob("*")]
assert len(validation_predictions) == 1
validation_predictions = validation_predictions[0]

validation_prediction = []
with open(validation_predictions, "rb") as file:
    data = pickle.load(file)
    for k,v in data.items():
        validation_prediction += [v]

In [ ]:
print(np.asarray(validation_prediction).shape)
print(np.asarray(validation_labels).shape)

In [ ]:
sys.path.insert(0, "EPFD")
from main import *

In [ ]:
def accuracy(l,p):
    return np.sum(np.equal(l,p))/l.shape

def evaluate(solution_set, data, labels):
    solutions = []
    for model in solution_set:
        solutions += [data[model]]
    solution = np.median(np.asarray(solutions), axis=0)
    return accuracy(labels, solution)

In [ ]:
from argparse import Namespace
config = wandb.config

lam = config.lambda
nb_pru = config.nb_pru
name_pru = config.name_pru

setting = {"nb_pru" : nb_pru, "name_pru": name_pru, "distributed": False, "lam": lam, "m":2}
args = Namespace(**setting)
indices = main(args, validation_labels, validation_prediction)
solution_set = np.asarray(list(data.keys()))[indices]
print(setting)
print(solution_set)
acc = evaluate(solution_set, data, validation_labels)
print(acc)
wandb.log({"accuracy": acc})